# This module computes the the Weyl scalars based on Baker, Campanelli, and Lousto. PRD 65, 044001 (2002);
## https://arxiv.org/abs/gr-qc/0104063

In [3]:
# Step 1: import all needed modules from NRPy+:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
import reference_metric as rfm
rfm.reference_metric()
from outputC import *
import BSSN_RHSs as bssn
import sympy as sp


initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::RMAX


ModuleNotFoundError: No module named 'BSSN_RHSs'

In [ ]:
# Step 1: Initialize WeylScalar parameters
thismodule = __name__
# Use proper names for Tetrad Choices. If no name given (hunt the literature), then use the literature reference as the name.
TetradChoice = par.initialize_param(par.glb_param("char", thismodule, "TetradChoice", "Approx_QuasiKinnersley"))
# Why are these needed? # They were used in the Mathematica version, but even there, they didn't do much.
# xorig = par.initialize_param(par.glb_param("REAL", thismodule, "xorig", "0.0"))
# yorig = par.initialize_param(par.glb_param("REAL", thismodule, "yorig", "0.0"))
# zorig = par.initialize_param(par.glb_param("REAL", thismodule, "zorig", "0.0"))
# offset = par.initialize_param(par.glb_param("REAL", thismodule, "offset", "1.0e-15"))



We will now define the Levi-Civita symbol, $\epsilon_{ijk}$. This function, when called, will initialize the tensor and set all the components to 1 (for even permutations of the indices), -1 (for odd permutations), or zero (if any indices are repeated).

In [ ]:
# Step 2: Define the Levi-Civita symbol. Amongst other uses, this is needed for the construction of the approximate quasi-Kinnersley tetrad. <- better description needed.
def define_LeviCivitaSymbol(DIM=-1):
    if DIM == -1:
        DIM = par.parval_from_str("DIM")

    LeviCivitaSymbol = ixp.zerorank3()

    for i in range(DIM):
        for j in range(DIM):
            for k in range(DIM):
                # From https://codegolf.stackexchange.com/questions/160359/levi-civita-symbol :
                LeviCivitaSymbol[i][j][k] = (i - j) * (j - k) * (k - i) / 2
    return LeviCivitaSymbol

We will now begin begin to compute the Weyl scalars themselves. We start by defining our choice of tetrad. (For now, only an approximate quasi-Kinnersley tetrad is available, but we plan to add more in the future.) This choice is drawn from https://arxiv.org/pdf/gr-qc/0104063.pdf

We begin with the vectors given in eqs. 5.6 and 5.7 of the paper,
\begin{align}
    v_1^a &= [-y,x,0] \\
    v_2^a &= [x,y,z] \\
    v_3^a &= {\rm det}(g)^{1/2} g^{ad} \epsilon_{dbc} v_1^b v_2^c,
\end{align}
and carry out the Gram-Schmidt orthonormalization process. The vectors $w_i^a$ are placeholders; the final product of the orthonormalization is the vectors $e_i^a$.

In [ ]:
# Step 3: Compute the Weyl scalars
def WeylScalars():
    # Step 1:
    bssn.BSSN_RHSs()

    # Step 2b: Set spatial dimension (must be 3 for BSSN)
    DIM = 3
    par.set_parval_from_str("grid::DIM",DIM)

    x = rfm.xxCart[0]
    y = rfm.xxCart[1]
    z = rfm.xxCart[2]

    if TetradChoice == "Approx_QuasiKinnersley":
        # Eqs 5.6 in https://arxiv.org/pdf/gr-qc/0104063.pdf
        xmoved = x# - xorig  # Make sure I'm handling coordinates correctly
        ymoved = y# - yorig
        zmoved = z# - zorig

        # Eqs 5.7
        v1U = ixp.zerorank1("v1U")
        v2U = ixp.zerorank1("v2U")
        v3U = ixp.zerorank1("v3U")
        v1U[0] = -ymoved
        v1U[1] = xmoved + offset
        v1U[2] = 0
        v2U[0] = xmoved + offset
        v2U[1] = ymoved
        v2U[2] = zmoved
        LeviCivitaSymbol = define_LeviCivitaSymbol()
        for a in range(DIM):
            for b in range(DIM):
                for c in range(DIM):
                    for d in range(DIM):
                        v3U[a] += sp.sqrt(bssn.detgammabar) * bssn.gammabarUU[a][d] * LeviCivitaSymbol[d][b][c] * v1U[b] *v2U[c]

        # Gram-Schmidt orthonormalization of the tetrad.
        # The w* vectors here are used to temporarily hold values on the way to the final vectors e*

        w1U = ixp.zerorank1("w1U")
        for a in range(DIM):
            w1U[a] = v1U[a]
        omega11 = 0
        for a in range(DIM):
            for b in range(DIM):
                omega11 += w1U[a] * w1U[b] * bssn.gammabarDD[a][b]
        e1U = ixp.zerorank1("e1U")
        for a in range(DIM):
            e1U[a] = w1U[a] / sp.sqrt(omega11)

        omega12 = 0
        for a in range(DIM):
            for b in range(DIM):
                omega12 += e1U[a] * v1U[b] * bssn.gammabarDD[a][b]
        w2U = ixp.zerorank1("w2U")
        for a in range(DIM):
            w2U[a] = v2U[a] - omega12*e1U[a]
        omega22 = 0
        for a in range(DIM):
            for b in range(DIM):
                omega22 += w2U[a] * w2U[b] *bssn.gammabarDD[a][b]
        e2U = ixp.zerorank1("e2U")
        for a in range(DIM):
            e2U[a] = w2U[a] / sqrt(omega22)

        omega13 = 0
        for a in range(DIM):
            for b in range(DIM):
                omega13 += e1U[a] * v3U[b] * bssn.gammabarDD[a][b]
        omega23 = 0
        for a in range(DIM):
            for b in range(DIM):
                omega23 += e2U[a] * v3U[b] * bssn.gammabarDD[a][b]
        w3U = ixp.zerorank1("w3U")
        for a in range(DIM):
            w3U[a] = v3U[a] - omega13*e1U[a] - omega23*e2U[a]
        omega33 = 0
        for a in range(DIM):
            for b in range(DIM):
                omega33 += w3U[a] * w3U[b] * bssn.gammabarDD[a][b]
        e3U = ixp.zerorank1("e3U")
        for a in range(DIM):
            e3U[a] = w3U[a] / sqrt(omega33)

Now that we have orthogonal, normalized vectors, we can contruct the tetrad itself, again drawing on eqs. 5.6. We can draw on SymPy's built-in tools for complex numbers to build the complex vectors $m$ and $\bar{m}$:
\begin{align}
    l^a &= \frac{1}{\sqrt{2}} e_2^a \\
    n^a &= \frac{-1}{\sqrt{2}} e_2^a \\
    m^a &= \frac{1}{\sqrt{2}} (e_3^a + i e_1^a) \\
    \bar{m}^a &= \frac{1}{\sqrt{2}} (e_3^a - i e_1^a)
\end{align}

We will also assume that $n^0 = \frac{1}{\sqrt{2}}$ and that $m^0 = \bar{m}^0 = 0$. This last assumption in particular will significantly reduce the terms needed to find $\psi_4$

In [ ]:
        # Construct the tetrad
        # Eqs. 5.6
        isqrt2 = 1/sp.sqrt(2)
        ltetU = isqrt2 * e2U
        ntetU = -isqrt2 * e2U
        mtetU = isqrt2 * (e3U + sp.I*e1U)
        mtetbarU = sp.conjugate(mtetU)
        nn = isqrt2

Now that we have the tetrad in place, we can contract it with the Weyl tensor to obtain the Weyl scalars. Naturally, we must first construct the Weyl tensor to do that. We will first build the Riemann curvature tensor,
\begin{align}
R_{abcd} = \frac{1}{2} (\bar{\gamma}_{ad,cb}+\bar{\gamma}_{bc,da}-\bar{\gamma}_{ac,bd}-\bar{\gamma}_{bd,ac}) + \bar{\gamma}_{je}\bar{\Gamma}^{j}_{bc}\bar{\Gamma}^{e}_{ad} - \bar{\gamma}_{je}\bar{\Gamma}^{j}_{bd}\bar{\Gamma}^{e}_{ac}
\end{align}
since several terms in our expression for $\psi_4$ are contractions of this tensor.
To do this, we need second derivatives of the metric tensor, $\bar{\gamma}_{ab,cd}$.

We also need the extrinsic curvature tensor $K_{ij}$; it can be shown that 
\begin{align}
K_{ij} &= e^{4 \phi} (\bar{A}_{ij} + \frac{1}{3} \bar{\gamma}_{ij} K)
\end{align}

In [ ]:
    # Declare the second derivative of the metric.
    gammarbarDD_dDD = ixp.declarerank4("gammabarDD_dDD","sym12_sym34")
    # Declare and construct the Riemann curvature tensor:
    RiemannDDDD = ixp.zerorank4()
    for a in range(DIM):
        for b in range(DIM):
            for c in range(DIM):
                for d in range(DIM):
                    RiemannDDDD[a][b][c][d] = (bssn.gammabarDD_dDD[a][d][c][b] + \
                                               bssn.gammabarDD_dDD[b][c][d][a] - \
                                               bssn.gammabarDD_dDD[a][c][b][d] - \
                                               bssn.gammabarDD_dDD[b][d][a][c]) / 2
                    for e in range(DIM):
                        for j in range(DIM):
                            RiemannDDDD[a][b][c][d] +=  bssn.gammabarDD[j][e] * bssn.GammabarUDD[j][b][c] * bssn.GammabarUDD[e][a][d] - \
                                                        bssn.gammabarDD[j][e] * bssn.GammabarUDD[j][b][d] * bssn.GammabarUDD[e][a][c]
    # The Ricci tensor was built by BSSN_RHSs.py
    # We also need the extrinsic curvature tensor. This can be built from quantities in BSSN_RHSs.py
    extrinsicKDD = ixp.zerorank2()
    for i in range(DIM):
        for j in range(DIM):
            extrinsicKDD[i][j] = (bssn.AbarDD[i][j] + bssn.gammabarDD[i][j]*trK/3)/bssn.exp_m4phi

Baker, Campanelli, and Lousto showed that 
\begin{align}
\psi_4 =& (R_{ijkl} + 2K_{i[k}K_{l]j}) n^i \bar{m}^j n^k \bar{m}^l \\
&- 8 (K_{j[k,l]} + \Gamma^p_{j[k} K_{l]p}) n^{[0} \bar{m}^{j]} n^k \bar{m}^l \\
&+ 4 (R_{jl} - K_{jp} K^p_l + KK_{jl}) n^{[0} \bar{m}^{j]} n^{[0} \bar{m}^{l]}
\end{align}

In [ ]:
    GaussDDDD = ixp.zerorank4()
    for i in range(DIM):
        for j in range(DIM):
            for k in range(DIM):
                for l in range(DIM):
                    GaussDDDD[i][j][k][l] = RiemannDDDD[i][j][k][l] + extrinsicKDD[i][k]*extrinsicKDD[l][j] - extrinsicKDD[i][l]*extrinsicKDD[k][j]
    
    # Codazzi equation: involving covariant derivatives of the extrinsic curvature. We will first need to declare derivatives of extrinsicKDD
    extrinsicKDD_dD = ixp.declarerank3("extrinsicKDD_dD","sym12")
    CodazziDDD = ixp.zerorank3()
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                CodazziDDD[j][k][l] = extrinsicKDD_dD[j][l][k] - extrinsicKDD_dD[j][k][l]
                for p in range(DIM)
                    CodazziDDD[j][k][l] += bssn.GammabarUDD[p][j][l]*extrinsicKDD[k][p] - bssn.GammabarUDD[p][j][k]*extrinsicKDD[l][p]
                    
    # This next tensor might relate to the Mainardi eq.
    RojoDD = ixp.zerorank2()
    for j in range(DIM):
        for l in range(DIM):
            RojoDD[j][l] = bssn.RbarDD[j][l]
            for p in range(DIM):
                for d in range(DIM):
                    RojoDD[j][l] += extrinsicKDD[j][p]*bssn.gammabarUU[p][d]*extrinsicKDD[d][l] - trK*extrinsicKDD[j][l]
                    
    # Now we can calculate $\psi_4$ itself!
    psi4 = 0
    for i in range(DIM):
        for j in range(DIM):
            psi4 += RojoDD[j][l] * nn * nn * mtetbarU[j] * mtetbarU[l]
            for k in range(DIM):
                psi4 += 2 * CodazziDDD[j][k][l] * ntetU[k] * nn * mtetbarU[j] * mtetbarU[l]
                for l in range(DIM):
                    psi4 += GaussDDDD[i][j][k][l] * ntetU[i] * ntetU[k] * mtetbarU[j] * mtetbarU[l]
    